In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors

%matplotlib widget 

Environment closely follows OpenAI gym API. Currently can not be invoked with ```gym.make("env_id")```, though it should be easy to do.

In [ ]:
import env
checkers_env = env.Env()

A state is ```(5+NUMBER_OF_PIECES, BOARD_SIZE, BOARD_SIZE) ndarray```

Where

* ```state[0]``` — black pieces
* ```state[1]``` — white pieces
* ```state[2]``` — pieces ids
* ```state[3]``` — current turn (blacks=1, whites=0)
* ```state[4]``` — whether game in terminal state
* ```state[5+i]``` — allowed moves for piece number i

I should probably move allowed moves elsewhere but I have a lot of RAM to keep things unoptimized.


In [ ]:
""" Various plotting helper methods """

def show_state(state):
    plt.imshow(state)
    plt.show()

def show_board(board):
   cmap = colors.ListedColormap(['white', 'red'])
   bounds=[0,0.5,18]
   norm = colors.BoundaryNorm(bounds, cmap.N)
   # plt.figure(figsize=(4,4))
   plt.imshow(board, cmap=cmap, norm=norm, interpolation='none')
   plt.xticks(np.arange(0.5,8.5), [])
   plt.yticks(np.arange(0.5,8.5), [])

   plt.grid()

def do_the_flip(arr):
    return np.flip(np.flip(np.flip(arr, 0), 0), 1)

def show_trajectory_item(trajectory, index):
    board_i = monte_carlo_tree.state_to_board(trajectory[index].original_state)
    show_board(do_the_flip(board_i[0] + board_i[1]))

## Random Play Tree

This is very basic algorithm that plays the game by making random moves. Sometimes it reaches the end goal, but overall it supper inneficient.

In [ ]:
import monte_carlo_tree


def random_play():
    '''
    Play a game using random tree strategy
    '''
        
    tree = monte_carlo_tree.RandomPlayTree(checkers_env, 8)
    
    root_node = tree.root_node
    return tree.simulate(root_node)
    

def build_stats(playfunc, n_games=100):
    '''
    Play a number of random games and display result
    '''

    black_wins = 0
    white_wins = 0
    draws = 0
    moves = []
    
    for _ in range(n_games):
        m = playfunc()
        if m.value > 0:
            black_wins += 1
        elif m.value < 0:
            white_wins += 1
        elif m.value == 0:
            draws += 1
       
        moves.append(m.depth())
    
    print("Blacks: ", black_wins, "Whites: ", white_wins, "Draws: ", draws, "Moves mean:", np.mean(moves))

In [ ]:
build_stats(random_play, 10)

## Monte Carlo Search Tree

In [ ]:
from monte_carlo_tree import MonteCarloPlayTree


def mtsc_play():
    '''
    Play a game using MonteCarloSearchTree
    '''

    mcst = MonteCarloPlayTree(checkers_env, 8)
    
    root_node = mcst.root_node
    terminal_node = mcst.simulate(root_node)
    
    return terminal_node


In [ ]:
build_stats(mtsc_play, 10)

## Guided Tree Search

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

BOARD_SIZE = 8

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ActorCritic(nn.Module):

    def __init__(self, board_size=BOARD_SIZE):
        super(ActorCritic, self).__init__()
        
        self.board_size = BOARD_SIZE
        self.fc = nn.Linear(self.board_size**2, 256)

        # Policy head
        self.fc_action1 = nn.Linear(256, 128)
        self.fc_action2 = nn.Linear(128, self.board_size**4)
        
        # Critic head
        self.fc_value1 = nn.Linear(256, 128)
        self.fc_value2 = nn.Linear(128, 1)
        
    def forward(self, x, moves):
        
        y = x.view(-1, self.board_size**2)
        y = F.relu(self.fc(y))
        
        # Actor head
        a = F.relu(self.fc_action1(y))
        a = self.fc_action2(a)
        
        # availability of moves
        avail = moves.view(-1, self.board_size**4) 

        # locations where actions are not possible, we set the prob to zero
        maxa = torch.max(a)
        # subtract off max for numerical stability (avoids blowing up at infinity)
        exp = avail*torch.exp(a-maxa)
        prob = (exp/torch.sum(exp))
        
        prob = prob.view(-1, BOARD_SIZE,BOARD_SIZE,BOARD_SIZE,BOARD_SIZE)
        
        # Critic head
        value = torch.relu(self.fc_value1(y))
        value = torch.tanh(self.fc_value2(value))
        value = value.view(-1, 1)

        return prob, value

actor_critic_network = ActorCritic().to(device)

In [ ]:
tree = monte_carlo_tree.GuidedMonteCarloPlayTree(checkers_env, BOARD_SIZE, actor_critic_network, device)

In [ ]:
tree.train(10)

In [ ]:
torch.save(actor_critic_network.state_dict(), "./../assets/actor_critic.pt")